In [51]:
#!pip install qiskit-nature[pyscf] -U

## FROM QISKIT TUTORIAL: https://qiskit.org/ecosystem/nature/tutorials/03_ground_state_solvers.html

In [52]:
#!pip show qiskit_nature 
#!pip show pyscf

#Soln found here: https://quantumcomputing.stackexchange.com/questions/26927/help-on-pyscf-library

In [ ]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver

from qiskit_nature.second_q.mappers import JordanWignerMapper

from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver

from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.algorithms.optimizers import SLSQP
from qiskit.primitives import Estimator
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD

from qiskit_nature.second_q.algorithms import GroundStateEigensolver

from qiskit.circuit.library import TwoLocal

In [32]:
# Creating PySCF driver for respective molecule based on xyz file format 
# Specifically for Nitrogen (N2) molecule:
driver = PySCFDriver(
    atom="H 0 0 -0.37; H 0 0 0.37", ## H2 <Just Right>; charge/spin:0/0
    #atom="N 0 0 0; N 1.098 0 0", ##N2 <TOO LONG>; charge/spin:0/0
    #atom="N 0 0 -0.069; H 0 0.943 0.321; H -0.817 -0.472 0.321; H 0.817 -0.472 0.321", ##NH3 <TOO LONG>; charge/spin:0/0
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

# Creating the Electronic Structure (es) problem
es_problem = driver.run()

In [33]:
# Transforms the qubits (from initial '0' state to representing H_el in 2nd quantization form)
mapper = JordanWignerMapper()

In [34]:
# Classical routine
numpy_solver = NumPyMinimumEigensolver()

In [35]:
# Ansatz: Parameterized Quantum Circuit (Theta)
# Typically, we use UCCSD (unitary coupled cluster), which is chemistry-inspired
ansatz = UCCSD(
    es_problem.num_spatial_orbitals,
    es_problem.num_particles,
    mapper,
    initial_state=HartreeFock(  #HF provided cheap yet good starting approx. value 
        es_problem.num_spatial_orbitals,
        es_problem.num_particles,
        mapper,
    ),
)

# Estimator = Qiskit's primitive machine categorization; more suitable for VQE 
# other types is Sampler (which is used in other scenarios)
vqe_solver = VQE(Estimator(), ansatz, SLSQP())

# Initial parameter values 
vqe_solver.initial_point = [0.0] * ansatz.num_parameters

In [47]:
# Custom TwoLocal parameterized quantum circuit using:
    # Hadamard and RX gates for rotation/superposition
    # Controlled-Z gates for 'full' entanglement
    # 2 repetitions/layers
    
tl_circuit = TwoLocal(
    rotation_blocks=["h", "rx"],
    entanglement_blocks="cz",
    entanglement="full",
    reps=3,
    parameter_prefix="y",
)

# Second VQE (this time not using UCCSD)
vqe_solver2 = VQE(Estimator(), tl_circuit, SLSQP())

#### VQE vs. VQE2 vs. Classical-Solver

In [50]:
print("-----FROM VQE 1-----\n")

# Using VQE and JW-mapper in classical routine (i.e., GroundStateSolver) to find ground state energy of molecule
calc = GroundStateEigensolver(mapper, vqe_solver)

result_vqe1 = calc.solve(es_problem)
print(result_vqe1)

-----FROM VQE 1-----

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1.852388173512
  - computed part:      -1.852388173512
~ Nuclear repulsion energy (Hartree): 0.715104339081
> Total ground state energy (Hartree): -1.137283834431
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  0.0]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  0.00000013119]
    - computed part:      [0.0  0.0  0.00000013119]
  > Dipole moment (a.u.): [0.0  0.0  -0.00000013119]  Total: 0.00000013119
                 (debye): [0.0  0.0  -0.000000333452]  Total: 0.000000333452
 


In [49]:
print("-----FROM VQE 2-----\n")

# Using VQE and JW-mapper in classical routine (i.e., GroundStateSolver) to find ground state energy of molecule
calc = GroundStateEigensolver(mapper, vqe_solver2)

result_vqe2 = calc.solve(es_problem)
print(result_vqe2)

-----FROM VQE 2-----

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1.8523874571
  - computed part:      -1.8523874571
~ Nuclear repulsion energy (Hartree): 0.715104339081
> Total ground state energy (Hartree): -1.137283118019
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: -0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  0.0]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  -0.000000544446]
    - computed part:      [0.0  0.0  -0.000000544446]
  > Dipole moment (a.u.): [0.0  0.0  0.000000544446]  Total: 0.000000544446
                 (debye): [0.0  0.0  0.000001383845]  Total: 0.000001383845
 


In [44]:
print("-----CLASSICAL ONLY-----\n")

# GroundStateEigensolver taking only classical algorithm to find GSE
calc = GroundStateEigensolver(mapper, numpy_solver)
result_cls = calc.solve(es_problem)
print(result_cls)

-----CLASSICAL ONLY-----

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1.85238817357
  - computed part:      -1.85238817357
~ Nuclear repulsion energy (Hartree): 0.715104339081
> Total ground state energy (Hartree): -1.137283834488
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  0.0]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  0.0]
    - computed part:      [0.0  0.0  0.0]
  > Dipole moment (a.u.): [0.0  0.0  0.0]  Total: 0.0
                 (debye): [0.0  0.0  0.0]  Total: 0.0
 
